# Exercise session 1: Overview of the Xpress Python Interface

## Creating a simple LP

First, create the simple mixed integer linear program:

$$
\begin{array}{lll}
  \min & x + y\\
  \textrm{s.t.} & 2x + 3y \ge 6\\
                & 4x + 2y \ge 7\\
                & x,y \ge 0
\end{array}
$$

In [ ]:
# Location of manual and examples can be found by calling appropriate functions:

import xpress
print(xpress.manual())
print(xpress.examples())

In [ ]:
import xpress as xp

# TODO Declare variables: x = xp.var(...)

# TODO Define constraints: constr1 = 2*x + ...

# TODO Define objective: obj = ...

# TODO Create problem: p = xp.problem(...)

p.solve()

print(f'solution: x={p.getSolution(x)}, y={p.getSolution(y)}')

Add the constraint $x + 2y \le 2$ to the problem, then re-solve. The problem should now be infeasible.

In [ ]:
# TODO Create constraint

# TODO Add constraint to problem

# TODO Call solve() again

Use the `iisall` function to understand what made the problem infeasible. If needed, type `help(p.iisall)` for its docstring.

In [ ]:
# TODO Call iisall() on problem p (easy...).

Now delete the constraint (using the `problem.delConstraint` method) and, just to avoid the same solve as before, change the objective function coefficient of x to 4. Then re-solve and print primal and dual solution using `problem.getSolution` for the primal solution and `problem.getDual` for the dual solution. Recall that these functions can be called without arguments to obtain the whole vector, or with a variable/constraint object to get one or more values.

In [ ]:
# TODO Use delConstraint() to delete the last added constraint

# TODO Use p.chgobj() to change one objective coefficient.
# Its syntax has a list of variables for which the
# coefficient is changed and a list of corresponding 
# new coefficient, so the call is like p.chgobj([x1, x2, ...], [c1, c2, ...])

p.solve()
print(f'Solution: x={p.getSolution(x)}, y={p.getSolution(y)}')
print(f'Dual: constr1->{p.getDual(constr1)}, constr2->{p.getDual(constr2)}')

## Knapsack problem

Formulate and solve a knapsack problem with the following value/weight vectors:

$$
\begin{array}{lllrrrrrrr}
v& =& (12,&15,& 9,&11,& 8,& 7,&5)\\
w& =& (13, &18,& 9,& 12,& 8,& 10,& 4)
\end{array}
$$

and with knapsack capacity $C=40$. You will need the `xpress.Sum` operator.

In [ ]:
import xpress as xp

v = [12,15,9,11,8, 7,5]
w = [13,18,9,12,8,10,4]
C = 40

# Number of variables
n = len(v)

# TODO Define a list of n binary variables

# TODO Define the knapsack constraint and its objective

# TODO Create a problem and add variable list, constraint, and objective

p.solve()

print(p.getSolution())

## Reading and solving a MIP

Let's try to solve something bigger, like a file from the MIPLIB 2017 collection. Specifically, `roi2alpha3n4`. It should be available in the directory with this exercise session; load it and solve it as a Python problem.

In [ ]:
# TO BE DONE IN CLASS



Add a new constraint: the norm of the variable vector is at most 10. Even if the problem was read from a file, you can get the vector of all variables with `problem.getVariable`.

Note that before the constraint is added we'll have to postsolve the problem (using the `problem.postsolve()` function) as, for reasons of performance, the problem remains in a presolved state after the `solve()` call.

In [ ]:
import numpy as np
p.postsolve()

# TODO: Get all variables as a numpy array (with appropriate numpy dtype) using the getVariable()
# function wrapped in a np.array() call, i.e., np.array(p.getVariable(...), dtype=xp.npvar)

# TODO: Add constraint with sum of squares of all variables at most 100 (i.e. norm <= 10)

# Set a time limit and solve the problem
p.controls.maxtime=-30
p.solve()

With a real MIP at hand, how about we try to see the structure of the coefficient structure from a graphical standpoint? Use `matplotlib` to draw a density chart.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse

p.postsolve()

# Extract all coefficient data with getrows()

# mclind contains variable objects, not indices.
# Extract the index for all of them with getIndex()

# Use the spy() function for plotting the matrix
plt.show()

### NumPy operators and arrays of variables/constraints

NumPy is an essential toolbox for Python users. The Xpress Python interface can handle (multi-)arrays of floats, variables, expressions, constraints as naturally as with lists thereof.

In [ ]:
import numpy as np


We can also rewrite the knapsack problem using NumPy constructs.

In [ ]:
import xpress as xp
import numpy as np

v = np.array([12,15,9,11,8, 7,5])
w = np.array([13,18,9,12,8,10,4])
C = 40

n = len(v)

# Create numpy array of n binary variables

# Create constraints and objective for the knapsack

# Create (maximization!) problem

p.solve()

print(p.getSolution())

**Exercise**: Create a random 20x30 numpy matrix $A$ and vectors ${\bf b}$ and ${\bf c}$, then create a problem with ${\bf c}$ as objective function coefficient vector and constraints $A{\bf x} \le {\bf b}$.

In [ ]:
n = 3 # number of variables
m = 2 # number of constraints

# TODO Create numpy random matrix A, b, and c using the function
# np.random.random(), with a single parameter k to create a
# 1-dimensional array and the tuple (k1,k2) for a matrix

# TODO Create a variable vector

# TODO Create objective and constraint using the xpress.Dot() operator.

# TODO Create problem

p.solve()

Let's spice it up: make the objective function quadratic, with a matrix $Q$ that is PSD but not necessarily diagonal. Make the objective

$$
(x-x_0)^T Q (x-x_0)
$$

In [ ]:
# This matrix is positive semidefinite, so that the problem is convex.
Q = np.random.random((n,n)) - 0.5 + 3*n*np.eye(n)

# Creates a uniformly random point with coordinates in [0,10]
x0 = 10 * np.random.random(n)  

# TODO Alter the objective function so as to minimize weighted
# (with Q) distance between x and x0: (x-x0)' Q (x-x0)

p.solve()

Obviously we can use arrays of variables in constraints. Let's consider the following problem:

### Production planning with quadratic cost

A factory must plan production on one machine for the upcoming $N=12$ days. If, on one day, the machine is on, it must pay a fixed cost F, regardless of how much is produced on that machine. If the machine is on, the production must range between L and U. It must also pay a cost that is C times the square of the produced amount.

The demand at a given day is $w_i$, and the unsold product can be stored in a storage container of capacity $M$. Finally, the storage level at the end of the $N$ days must be the same as the beginning, which is set as $S$.

**Model**. Define two classes of variables, $x$ (continuous) and $y$ (binary), indicating the amount of production in a given day and whether the machine is on or off, respectively. Also, introduce a storage variable $s$ that is also indexed by the set of periods.

In [ ]:
import xpress as xp

n = 12  # number of periods
N = range(n)

# Input parameters

C = 2   # proportional cost
F = 9000  # fixed cost
L = 10   # min. production level
U = 90   # max. production level
M = 170  # maximum storage level
S = 25   # initial storage level

# Demand
w = [60, 15, 25, 70, 70, 85, 10, 5, 65, 40, 50, 15]

# TODO Create variables: y is a binary vector of size n, x has size
# n, while s has size n+1 to include storage at time n + 1

# TODO Total production cost (to be minimized)

# TODO y determines bounds on x: if y=1, then x is
# between L and U, otherwise it is 0. How to do it
# using notions learned during the course? Hint: think
# about big-M.

# TODO Constrain storage at last period

# TODO Production balance:
# the storage at time i plus the production at time i
# must equal
# the storage at time i+1 plus the demand at time i.

# TODO create a problem with the variables, objectives, and constraints
# as defined above.

p.solve()

print('time demand machine productn storage')
for i in N:
    print('{0:4d} {1:6d} {2:7.0f} {3:8.4f} {4:7.4f}'.format(i, w[i],
                                                            p.getSolution(y[i]),
                                                            p.getSolution(x[i]),
                                                            p.getSolution(s[i])))

**Homework**: Rewrite the problem using NumPy for defining parameters, variables, constraints, and objective.

In [ ]:
import xpress as xp
import numpy as np

# Demand is translated to a NumPy array for using NumPy operators
w = np.array(w)

# TODO Variables: use vars() but remember to name them in order to
# avoid variables with same name. For instance, use x = xp.vars(..., name='prod')
# for production variables

# TODO Total production cost (to be minimized)

# TODO y determines bounds on x (just two numpy inequalities)

# TODO storage at last period (doesn't change from previous model)

# TODO Production balance: use the range operator of arrays for the s[] variable to write a 
# single numpy constraint.

# TODO create problem

p.solve()

print('time demand machine production storage')
for i in N:
    print('{0:4d} {1:6d} {2:7.0f} {3:8.4f} {4:7.4f}'.format(i, w[i],
                                                            p.getSolution(y[i]),
                                                            p.getSolution(x[i]),
                                                            p.getSolution(s[i])))